# **1. Informations matériel (GPU)**

In [ ]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Tue Jan 25 20:10:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **2. Importation des librairies**

In [ ]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2 
import csv
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.models import Sequential
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance
from google.colab import files
print("Tensorflow version: "+tf.__version__)
print("Keras version: " + tf.keras.__version__)
from sklearn.model_selection import train_test_split

Tensorflow version: 2.7.0
Keras version: 2.7.0


#**3. Téléchargement des données (radiographies Covid-19)"**

In [ ]:
!rm -rf sample_data
! wget https://smartappli.eu/Train_Test_Covid19.zip
! unzip Train_Test_Covid19.zip
! rm Train_Test_Covid19.zip

--2022-01-25 20:10:03--  https://smartappli.eu/Train_Test_Covid19.zip
Resolving smartappli.eu (smartappli.eu)... 51.254.94.183, 2001:41d0:301:12::20
Connecting to smartappli.eu (smartappli.eu)|51.254.94.183|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://smartappli.eu [following]
--2022-01-25 20:10:04--  https://smartappli.eu/
Reusing existing connection to smartappli.eu:443.
HTTP request sent, awaiting response... 302 Found
Location: https://smartappli.eu [following]
--2022-01-25 20:10:04--  https://smartappli.eu/
Reusing existing connection to smartappli.eu:443.
HTTP request sent, awaiting response... 302 Found
Location: https://smartappli.eu [following]
--2022-01-25 20:10:04--  https://smartappli.eu/
Reusing existing connection to smartappli.eu:443.
HTTP request sent, awaiting response... 302 Found
Location: https://smartappli.eu [following]
--2022-01-25 20:10:04--  https://smartappli.eu/
Reusing existing connection to smartappli.eu:443.
HTTP r

#**4. Création du fichier de labels "classes.txt"**

In [ ]:
!printf '%s\n' 'covid-19' 'normal' 'viral-pneumonia'> classes.txt

#**5. Paramètres d'entrainement et sélection du modèle pré-entrainé**

In [ ]:
nb_classes = 3
image_size=(224,224)
nbr_batch_size=8 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
dataset_name='New_DataSet_Covid19' #@param ["New_DataSet_Covid19","Dataset1","Dataset2","Dataset3"]
dataset_path = os.path.join(dataset_name,'train/')
dataset_test_path = os.path.join(dataset_name,'test/')
classes_path = "classes.txt"
epochs = 100 #@param {type:"slider", min:5, max:100, step:5}
classifier = "Xception" #@param ["Xception","VGG16","VGG19","ResNet50","ResNet121","ResNet152","ResNet50V2","ResNet121V2","ResNet152V2","InceptionV3","InceptionResNetV2","MobileNet","MobileNetV2","DenseNet121","DenseNet169","DenseNet201","NASNetLarge","NASNetMobile"] {type:"string"}
result_path = 'results/'+classifier

# **6. Fonction de génération des données à partir de chemins**

In [ ]:
def generate_from_paths_and_labels(input_paths, labels, batch_size, input_size=image_size):
    num_samples = len(input_paths)
    while 1:
        perm = np.random.permutation(num_samples)
        input_paths = input_paths[perm]
        labels = labels[perm]
        for i in range(0, num_samples, batch_size):
            inputs = list(map(
                lambda x: image.load_img(x, target_size=input_size),
                input_paths[i:i+batch_size]
            ))
            inputs = np.array(list(map(
                lambda x: image.img_to_array(x),
                inputs
            )))
            inputs = preprocess_input(inputs)
            yield (inputs, labels[i:i+batch_size])

# **7. Récupérer les images et leurs labels  (peut prendre quelques minutes)**

In [ ]:
def load_data(dataset_path, classes_path):
  with open(classes_path, 'r') as f:
    classes = f.readlines()
    classes = list(map(lambda x: x.strip(), classes))
  num_classes = len(classes)

  # Récupurer les images et les classes
  input_paths, labels = [], []
  for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    class_id = classes.index(class_name)
    for path in os.listdir(class_path):
        path = os.path.join(class_path, path)
        if imghdr.what(path) == None:
            # this is not an image file
            continue
        input_paths.append(path)
        labels.append(class_id)
  return input_paths, labels

In [ ]:
input_paths, labels = load_data(dataset_path, classes_path)

FileNotFoundError: ignored

# **8. Conversion des labels au format "One Hot" (A COMPLETER)**

In [ ]:
nb_classes = 3
labels = np_utils.to_categorical(labels, nb_classes)

#**9. Conversion des données d'entrée au formar np.array**

In [ ]:
input_paths = np.array(input_paths)

#**10. Permutation des données (A COMPLETER)**

In [ ]:
# shuffle dataset (permuter les données)
perm = np.random.permutation(len(input_paths))
labels = labels[perm]
input_paths = input_paths[perm]

#**11. Division des données : 80% train et 20% validation (A COMPLETER)**

In [ ]:
# split dataset for training and validation
border = 0.2                             
# train_labels, val_labels = #???????????????????????????????????????????????????
train_input_paths, val_input_paths, train_labels, val_labels = train_test_split(input_paths, labels, test_size = border, random_state=42)
print("Training on %d images and labels" % (len(train_input_paths)))
print("Validation on %d images and labels" % (len(val_input_paths)))

# **12. Modèle from Scractch  (A compléter)**

In [ ]:
nb_filters = 32
kernel_size = (3,3)
input_shape = (224,224,3)
pool_size = (2,2)

model = Sequential()
# Layer 1
model.add(Conv2D(nb_filters, kernel_size, input_shape= input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.3))
# Layer 2
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Conv2D(nb_filters, kernel_size, input_shape= input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.1))


# Layer3
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
# Layer 4
model.add(Dense(128))
model.add(Activation('relu'))



# Output Layer
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# **13. Téléchargement d'un modèle préentraîné et Transfer Learning (A compléter)**

In [ ]:
base_model = tf.keras.applications.ResNet50(include_top = False, weights='imagenet', input_shape=input_shape)

# create a custom top classifier
x=base_model.output
x = GlobalAveragePooling2D()(x)
x=Dense(1028, activation ='relu')(x)
predictions = Dense(nb_classes, activation='softmax')(x)
model = Model(inputs=base_model.inputs, outputs = predictions)

# **14. Entraînement du modèle (A compléter)**

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
stopper = EarlyStopping(monitor='val_loss',
                        min_delta=0.1,
                        patience=3,
                        verbose=1,
                        mode='auto')
ckpt_save = os.path.join("./results/classifier", 'model_fine_ep{epoch}_valloss{val_loss:.3f}.h5')
checkpoint=ModelCheckpoint(ckpt_save, monitor="val_loss", verbose=1, save_best_only=True, mode='auto')
callback_list=[checkpoint, stopper]

In [ ]:
# pour permettre le ré-entrainement des couches
for layer in model.layers:
    layer.trainable = True

# recompiler le modèle
tf.keras.backend.clear_session()   # Supprimer les poids du dernier entrainement
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy']) # Remplacer les *** par une optimiseur 

# Création du dossier pour sauvegrader le model
if os.path.exists(result_path) == False:
    os.makedirs(result_path)

history=model.fit_generator(
    generator=generate_from_paths_and_labels(
        input_paths=train_input_paths,
        labels=train_labels,
        batch_size=nbr_batch_size,
        input_size=input_shape # Adapter suivant le modèle
        
    ),
    steps_per_epoch=math.ceil(len(train_input_paths) / nbr_batch_size),
    epochs=epochs,
    validation_data=generate_from_paths_and_labels(
        input_paths=train_input_paths,
        labels=train_labels,
        batch_size=nbr_batch_size,
        input_size=input_shape
    ),
    validation_steps=math.ceil(len(val_input_paths) / nbr_batch_size),
    callbacks=callback_list,
    verbose=1
)

#**15. Sauvegarder me modèle final (A COMPLETER)**


In [ ]:
model.save("./results/test.h5")

#**16. Affichage des courbes d'entrainement/validation**

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

# **17. Récupérer les données de test**

In [ ]:
test_paths, test_labels = load_data(dataset_test_path, classes_path)

# **18. Préparer les données de test**

In [ ]:
test_labels=to_categorical(test_labels,nb_classes)
test_paths = np.array(test_paths)

# **19. Calculer le score de test (A compléter)**

In [ ]:
score = model.evaluate_generator(
    generate_from_paths_and_labels(
      input_paths= val_input_paths,
      labels=val_labels,
      batch_size=128,
      input_size=input_shape),
    steps=(len(test_paths)/128),workers = 1)

print ("Test images number:", len(test_paths))

print("Test %s: %.2f%%" % (model.metrics_names[0], score[0]))
print("Test %s: %.2f%%" % (model.metrics_names[1], score[1]*100))